<a href="https://colab.research.google.com/github/IhkafAmeny/HSSF_Implementation/blob/main/HSSFAbideFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models

In [ ]:
CSV_PATH = '/kaggle/input/abide1000/phenotypes_clean1.csv'
IMAGES_DIR = '/kaggle/input/abide1000/image'
K = 5  # nombre de clients

class AbideDataset(Dataset):
    def __init__(self, csv_path, images_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.images_dir = images_dir
        self.transform = transform
        self.label_map = {'control': 0, 'autism': 1}

        # Ajout du label numérique
        self.data['label'] = self.data['diagnosis'].map(self.label_map)

        # Nettoyage des labels manquants
        self.data = self.data.dropna(subset=['label']).reset_index(drop=True)

        # Liste des fichiers d'image existants
        available_images = set(os.listdir(images_dir))

        # Filtrage : garder uniquement les lignes dont l'image existe
        self.data = self.data[
            self.data['subject_id'].apply(lambda sid: f"{sid}.png" in available_images)
        ].reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        subject_id = self.data.iloc[idx]['subject_id']
        label = int(self.data.iloc[idx]['label'])
        img_path = os.path.join(self.images_dir, f"{subject_id}.png")
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_abide_dataloaders(batch_size=4):
    dataset = AbideDataset(CSV_PATH, IMAGES_DIR, transform=transform)
    indices = np.arange(len(dataset))
    np.random.shuffle(indices)

    public_size = int(0.2 * len(dataset))
    client_size = (len(dataset) - public_size) // K
    public_indices = indices[:public_size]
    client_indices = indices[public_size:]

    public_dataset = Subset(dataset, public_indices)
    public_loader = DataLoader(public_dataset, batch_size=batch_size, shuffle=True)

    unlabeled_loaders = []

    for i in range(K):
        start_idx = i * client_size
        # dernier client prend le reste
        end_idx = (i + 1) * client_size if i < K - 1 else len(client_indices)
        subset_indices = client_indices[start_idx:end_idx]
        subset = Subset(dataset, subset_indices)

        # Dataset qui masque les labels (pour apprentissage semi-supervisé)
        class UnlabeledSubset(Dataset):
            def __init__(self, subset):
                self.subset = subset
            def __len__(self):
                return len(self.subset)
            def __getitem__(self, idx):
                x, _ = self.subset[idx]
                return x, -1  # label fictif

        unlabeled_subset = UnlabeledSubset(subset)
        loader = DataLoader(unlabeled_subset, batch_size=batch_size, shuffle=True)
        unlabeled_loaders.append(loader)

    return public_loader, unlabeled_loaders

#  Modèles hétérogènes clients et serveur

def get_client_model(client_id):
    num_classes = 2
    if client_id in [0, 1, 2]:
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    elif client_id == 3:
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    else:  # client_id == 4
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model

def get_server_model():
    num_classes = 2
    model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model


# Modules SA et RPG

class SelfAssessmentModule:
    def estimate_confidence(self, logits):
        probs = F.softmax(logits, dim=1)
        confidence, _ = torch.max(probs, dim=1)
        return confidence

class RPGModule:
    def generate_pseudo_label(self, logits, confidence, threshold=0.8):
        pseudo_labels = torch.argmax(logits, dim=1)
        mask = confidence > threshold
        return torch.where(mask, pseudo_labels, torch.tensor(-1, device=logits.device))


#  Fonctions perte et accuracy

def supervised_loss(pred, target):
    return F.cross_entropy(pred, target)

def kl_divergence_loss(pred, soft_targets):
    pred_log_prob = F.log_softmax(pred, dim=1)
    soft_prob = F.softmax(soft_targets, dim=1)
    return F.kl_div(pred_log_prob, soft_prob, reduction='batchmean')

def accuracy(pred, target):
    preds = torch.argmax(pred, dim=1)
    correct = (preds == target).sum().item()
    return correct / target.size(0)


#  Classe Client

class Client:
    def __init__(self, id, model, unlabeled_loader, public_loader, device):
        self.id = id
        self.device = device
        self.model = model.to(self.device)
        self.unlabeled_loader = unlabeled_loader
        self.public_loader = public_loader
        self.sa = SelfAssessmentModule()
        self.rpg = RPGModule()
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-4)

    def train_supervised_on_public(self, epochs):
        self.model.train()
        for _ in range(epochs):
            for x, y in self.public_loader:
                x, y = x.to(self.device), y.to(self.device)
                self.optimizer.zero_grad()
                pred = self.model(x)
                loss = supervised_loss(pred, y)
                loss.backward()
                self.optimizer.step()

    def train_semi_supervised_on_local(self, epochs, pseudo_label_threshold=0.8):
        self.model.train()
        for _ in range(epochs):
            for x_unlabeled, _ in self.unlabeled_loader:
                x_unlabeled = x_unlabeled.to(self.device)
                self.optimizer.zero_grad()
                logits = self.model(x_unlabeled)
                confidence = self.sa.estimate_confidence(logits)
                pseudo_labels = self.rpg.generate_pseudo_label(logits, confidence, pseudo_label_threshold)
                valid_mask = pseudo_labels != -1
                if valid_mask.sum() == 0:
                    continue
                filtered_x = x_unlabeled[valid_mask]
                filtered_pseudo_labels = pseudo_labels[valid_mask]

                # Sup loss sur public loader (1 batch)
                loss_sup = 0
                for x_pub, y_pub in self.public_loader:
                    x_pub, y_pub = x_pub.to(self.device), y_pub.to(self.device)
                    pred_pub = self.model(x_pub)
                    loss_sup = supervised_loss(pred_pub, y_pub)
                    break

                pred_filtered = self.model(filtered_x)
                loss_unsup = supervised_loss(pred_filtered, filtered_pseudo_labels)
                loss = loss_sup + loss_unsup
                loss.backward()
                self.optimizer.step()

    def predict_logits_on_public(self):
        self.model.eval()
        all_logits = []
        with torch.no_grad():
            for x, _ in self.public_loader:
                x = x.to(self.device)
                logits = self.model(x)
                all_logits.append(logits.cpu())
        return torch.cat(all_logits, dim=0)

    def fuse_knowledge(self, server_logits, public_loader, epochs=1):
        self.model.train()
        for epoch in range(epochs):
            start_idx = 0
            for x, _ in public_loader:
                x = x.to(self.device)
                batch_size = x.size(0)
                logits_batch = server_logits[start_idx:start_idx + batch_size].to(self.device)
                start_idx += batch_size
                self.optimizer.zero_grad()
                client_pred = self.model(x)
                loss = kl_divergence_loss(client_pred, logits_batch)
                loss.backward()
                self.optimizer.step()

    def eval_accuracy_on_public(self):
        self.model.eval()
        total_acc = 0
        total_samples = 0
        with torch.no_grad():
            for x, y in self.public_loader:
                x, y = x.to(self.device), y.to(self.device)
                pred = self.model(x)
                total_acc += accuracy(pred, y) * x.size(0)
                total_samples += x.size(0)
        return total_acc / total_samples

#  HSSF  Boucle fédérée

def aggregate_logits(logits_list):
    return torch.stack(logits_list).mean(dim=0)

def federated_training(clients, server_model, public_loader, rounds=10, device=torch.device('cpu')):
    server_model.to(device)
    for round in range(rounds):
        print(f"\n Round {round + 1}")

        print(" Étape 1 : Supervised Learning (SA)")
        for client in clients:
            client.train_supervised_on_public(epochs=1)

        print(" Étape 2 : Regularity Condensation")
        all_logits = [client.predict_logits_on_public() for client in clients]
        fused_logits = aggregate_logits(all_logits)

        print(" Étape 3 : Regularity Fusion")
        for client in clients:
            client.fuse_knowledge(fused_logits, public_loader, epochs=1)

        print(" Étape 4 : Semi-Supervised Local Learning")
        for client in clients:
            client.train_semi_supervised_on_local(epochs=1)

        print(" Évaluation des clients")
        accuracies = []
        total_correct = 0
        total_samples = 0
        for client in clients:
            acc = client.eval_accuracy_on_public()
            print(f"Client {client.id} - Accuracy: {acc:.4f}")
            num_samples = len(client.public_loader.dataset)
            accuracies.append((acc, num_samples))
            total_correct += acc * num_samples
            total_samples += num_samples

        global_acc = total_correct / total_samples
        mean_acc = sum(acc for acc, _ in accuracies) / len(accuracies)

        #print(f"Global Accuracy (pondérée) : {global_acc:.4f}")
        print(f"Global Accuracy  : {mean_acc:.4f}")




#  TRAIN

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    public_loader, unlabeled_loaders = get_abide_dataloaders(batch_size=32)

    clients = []
    for i in range(K):
        model = get_client_model(i)
        client = Client(i, model, unlabeled_loaders[i], public_loader, device)
        clients.append(client)

    server_model = get_server_model().to(device)

    federated_training(clients, server_model, public_loader, rounds=100, device=device)


Using device: cuda

 Round 1
 Étape 1 : Supervised Learning (SA)
 Étape 2 : Regularity Condensation
 Étape 3 : Regularity Fusion
 Étape 4 : Semi-Supervised Local Learning
 Évaluation des clients
Client 0 - Accuracy: 0.5856
Client 1 - Accuracy: 0.6171
Client 2 - Accuracy: 0.5090
Client 3 - Accuracy: 0.6306
Client 4 - Accuracy: 0.5045
Global Accuracy  : 0.5694

 Round 2
 Étape 1 : Supervised Learning (SA)
 Étape 2 : Regularity Condensation
 Étape 3 : Regularity Fusion
 Étape 4 : Semi-Supervised Local Learning
 Évaluation des clients
Client 0 - Accuracy: 0.8784
Client 1 - Accuracy: 0.8649
Client 2 - Accuracy: 0.8874
Client 3 - Accuracy: 0.7162
Client 4 - Accuracy: 0.5586
Global Accuracy  : 0.7811

 Round 3
 Étape 1 : Supervised Learning (SA)
 Étape 2 : Regularity Condensation
 Étape 3 : Regularity Fusion
 Étape 4 : Semi-Supervised Local Learning
 Évaluation des clients
Client 0 - Accuracy: 0.9730
Client 1 - Accuracy: 1.0000
Client 2 - Accuracy: 0.9820
Client 3 - Accuracy: 0.9234
Client 4 